In [ ]:
#funtion to get the url for every MLB game on a given day
def box_score(year,month,day):
    if day < 10:
        r = requests.get('http://gd2.mlb.com/components/game/mlb/year_'+ str(year) +'/month_0' + str(month) +'/day_0' + str(day) +'/master_scoreboard.json')
    else:
        r = requests.get('http://gd2.mlb.com/components/game/mlb/year_'+ str(year) +'/month_0' + str(month) +'/day_' + str(day) +'/master_scoreboard.json')
  
    game_data = r.json()['data']['games']['game']
    games = []
    if len(game_data) > 25:
        games.append(game_data['game_data_directory'])
    else:
        for n in range(len(game_data)):
            if game_data[n]['status']['status'] != 'Postponed':
                try:
                    requests.get('http://gd2.mlb.com'+ game_data[n]['game_data_directory'] +'/boxscore.json').json()['data']['boxscore']['batting']
                    games.append(game_data[n]['game_data_directory'])
                except:
                    pass
    return games

#funtions to create a dictionary of all stats for each MLB game on a given day
def parse_boxscore(year,month,day):
    game_list = box_score(year,month,day)
    daily_stat_dict = {}
    for game in game_list:
        try:
            batter = requests.get('http://gd2.mlb.com'+ game +'/boxscore.json').json()['data']['boxscore']['batting']
            pitcher = requests.get('http://gd2.mlb.com'+ game +'/boxscore.json').json()['data']['boxscore']['pitching']

            for n in range(0,len(batter[0]['batter'])):
                daily_stat_dict[batter[0]['batter'][n]['id']] = {'name': batter[0]['batter'][n]['name_display_first_last'],
                                                                 'ab': int(batter[0]['batter'][n]['ab']),
                                                                 'h_b': int(batter[0]['batter'][n]['h']),
                                                                 'r': int(batter[0]['batter'][n]['r']),
                                                                 'hr': int(batter[0]['batter'][n]['hr']),
                                                                 'rbi': int(batter[0]['batter'][n]['rbi']),
                                                                 'sb': int(batter[0]['batter'][n]['sb']),
                                                                 'pos': batter[0]['batter'][n]['pos']
                                                                }


            for n in range(0,len(batter[1]['batter'])):
                daily_stat_dict[batter[1]['batter'][n]['id']] = {'name': batter[1]['batter'][n]['name_display_first_last'],
                                                                 'ab': int(batter[1]['batter'][n]['ab']),
                                                                 'h_b': int(batter[1]['batter'][n]['h']),
                                                                 'r': int(batter[1]['batter'][n]['r']),
                                                                 'hr': int(batter[1]['batter'][n]['hr']),
                                                                 'rbi': int(batter[1]['batter'][n]['rbi']),
                                                                 'sb': int(batter[1]['batter'][n]['sb']),
                                                                 'pos': batter[1]['batter'][n]['pos']
                                                                }

            for n in range(0,len(pitcher[0]['pitcher'])):
                daily_stat_dict[pitcher[0]['pitcher'][n]['id']] = {'name': pitcher[0]['pitcher'][n]['name_display_first_last'],
                                                                   'so': int(pitcher[0]['pitcher'][n]['so']),
                                                                   'h_p': int(pitcher[0]['pitcher'][n]['h']),
                                                                   'bb': int(pitcher[0]['pitcher'][n]['bb']),
                                                                   'out': int(pitcher[0]['pitcher'][n]['out']),
                                                                   'er': int(pitcher[0]['pitcher'][n]['er']),
                                                                   'pos': pitcher[0]['pitcher'][n]['pos']
                                                                  }
                if 'note' in pitcher[0]['pitcher'][n]:
                    if 'W' in pitcher[0]['pitcher'][n]['note']:
                        daily_stat_dict[pitcher[0]['pitcher'][n]['id']].update({'w': 1})
                    if '(S,' in pitcher[0]['pitcher'][n]['note']:
                        daily_stat_dict[pitcher[0]['pitcher'][n]['id']].update({'sv': 1})
            for n in range(0,len(pitcher[1]['pitcher'])):
                daily_stat_dict[pitcher[1]['pitcher'][n]['id']] = {'name': pitcher[1]['pitcher'][n]['name_display_first_last'],
                                                                   'so': int(pitcher[1]['pitcher'][n]['so']),
                                                                   'h_p': int(pitcher[1]['pitcher'][n]['h']),
                                                                   'bb': int(pitcher[1]['pitcher'][n]['bb']),
                                                                   'out': int(pitcher[1]['pitcher'][n]['out']),
                                                                   'er': int(pitcher[1]['pitcher'][n]['er']),
                                                                   'pos': pitcher[1]['pitcher'][n]['pos']
                                                                  }
                if 'note' in pitcher[1]['pitcher'][n]:
                    if 'W' in pitcher[1]['pitcher'][n]['note']:
                        daily_stat_dict[pitcher[1]['pitcher'][n]['id']].update({'w': 1})
                    if '(S,' in pitcher[1]['pitcher'][n]['note']:
                        daily_stat_dict[pitcher[1]['pitcher'][n]['id']].update({'sv': 1})
        except:
            pass
                
    return daily_stat_dict

#funtion to pull out the individual stats for a list of specific players on a given day
def player_stats(year,month,day,team_dataframe):
    today = parse_boxscore(year,month,day)
    stat_list= ['ab',
                'h_b',
                'r',
                'hr',
                'rbi',
                'sb',
                'w',
                'sv',
                'so',
                'h_p',
                'bb',
                'out',
                'er',
                'pos'
               ]
    for index, row in team_dataframe.iterrows():
        if row['id'] in today:
            for n in range(0,len(stat_list)):
                if stat_list[n] in today[row['id']]:
                    team_dataframe.loc[index, stat_list[n]] = today[row['id']][stat_list[n]]
    team_dataframe = team_dataframe.fillna(value = 0)
    return team_dataframe 

#function to calculate the points generated by a player on a given day
def point_calc(year,month,day,team_dataframe):
    player_points = player_stats(year,month,day,team_dataframe)
    for index, row in player_points.iterrows():
        player_points.loc[index, 'points'] = row['w']*6 + row['sv']*6 + row['out'] + row['so'] - (row['h_p'] + row['bb'] + row['er']*2) + row['hr']*6 + row['sb']*5 + row['h_b']*4 + row['r']*2 + row['rbi']*2 - row['ab']
        
    return player_points

#funtion to remove the accents from player names for easier data management
def rem_acnt(u_string):
    data = u_string.decode('utf-8')
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    return normal

#function that separates players and the points they generated into their eligible positions
def points(dataframe):
    points_dict = {}
    pos_list = []
    for i in set(dataframe.yahoo_pos):
        pos_list.append(i)
    teams = []
    for i in set(dataframe.Team):
        teams.append(i)
    for n in range(0,len(teams)):
        points_dict[teams[n]] = {'C': [],
                                 '1B': [],
                                 '2B': [],
                                 '3B': [],
                                 'SS': [],
                                 'OF': [],
                                 'SP': [],
                                 'RP': []
                                }

    for index, row in dataframe.iterrows():
        for i in points_dict[row['Team']]:
            if i in row['yahoo_pos']:
                points_dict[row['Team']][i].append([row['name'],row['yahoo_pos'],row['points']])
    
    def takethird(list):
        return list[2]
    
    pos = ['C','1B','2B','3B','SS','OF','SP','RP']
    
    for i in points_dict:
        for n in range(0,len(pos)):
            points_dict[i][pos[n]].sort(key = takethird, reverse = True)

    return points_dict

def pos_clean(x):
    x = str(x)
    if 'F' in x:
        x = 'OF'
    return x


